# YOLOv8 Training Notebook (Reproducible)

This notebook trains and evaluates a YOLOv8 detector and organizes the workflow into:
1) environment + imports, 2) dataset configuration, 3) sanity checks, 4) training, 5) evaluation + inference, and 6) export.

**Reproducibility note:** dependencies should be declared in `requirements.txt` (or `pyproject.toml`), not installed inside this notebook.


In [ ]:
import os

# Optional: pin a specific GPU (set to "" to let the framework decide)
os.environ["CUDA_VISIBLE_DEVICES"] = "0"


In [ ]:
import warnings
warnings.filterwarnings("ignore")

from pathlib import Path
import random

import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image, ImageOps

from ultralytics import YOLO

# Optional helpers for notebook display
from IPython.display import display


In [ ]:
# --- Environment diagnostics (recommended for appendices) ---
import sys, platform
import ultralytics

print("Python      :", sys.version.split()[0])
print("Platform    :", platform.platform())
print("Ultralytics :", ultralytics.__version__)


## Dataset configuration

Set `DATASET_ROOT` to the folder that contains `data.yaml` and YOLO-format splits, e.g.:

```
DATASET_ROOT/
  data.yaml
  train/images, train/labels
  valid/images, valid/labels
  test/images,  test/labels   (optional)
```


In [ ]:
# --- Paths (edit these two lines) ---
DATASET_ROOT = Path("/workspace")  # <- update to your dataset location
YAML_FILE = DATASET_ROOT / "data.yaml"

assert YAML_FILE.exists(), f"data.yaml not found: {YAML_FILE}"
print("YAML_FILE:", YAML_FILE)


In [ ]:
# Convenience paths
TRAIN_IMAGES = DATASET_ROOT / "train" / "images"
TRAIN_LABELS = DATASET_ROOT / "train" / "labels"
VAL_IMAGES   = DATASET_ROOT / "valid" / "images"
VAL_LABELS   = DATASET_ROOT / "valid" / "labels"

for p in [TRAIN_IMAGES, TRAIN_LABELS, VAL_IMAGES, VAL_LABELS]:
    assert p.exists(), f"Missing path: {p}"

print("Train images:", TRAIN_IMAGES)
print("Val images  :", VAL_IMAGES)


## Model initialization


In [ ]:
# Pretrained checkpoint (edit as needed)
PRETRAINED_WEIGHTS = Path("yolov8x.pt")  # relative path inside repo

model = YOLO(str(PRETRAINED_WEIGHTS))
model


## Dataset sanity checks
- counts of images/labels
- basic image size distribution
- random visual inspection


In [ ]:
def list_images(folder: Path):
    exts = {".png", ".jpg", ".jpeg", ".bmp", ".tif", ".tiff"}
    return sorted([p for p in folder.iterdir() if p.is_file() and p.suffix.lower() in exts])

train_imgs = list_images(TRAIN_IMAGES)
val_imgs   = list_images(VAL_IMAGES)

print(f"Train images: {len(train_imgs)}")
print(f"Val images  : {len(val_imgs)}")

print("Train labels:", len(list(TRAIN_LABELS.glob("*.txt"))))
print("Val labels  :", len(list(VAL_LABELS.glob("*.txt"))))


In [ ]:
def collect_image_shapes(paths, max_items=200):
    shapes = []
    for p in paths[:max_items]:
        im = cv2.imread(str(p), cv2.IMREAD_UNCHANGED)
        if im is not None:
            shapes.append(im.shape[:2])  # (H, W)
    return shapes

train_shapes = collect_image_shapes(train_imgs)
val_shapes   = collect_image_shapes(val_imgs)

def summarize_shapes(shapes, name):
    if not shapes:
        print(f"{name}: no readable images")
        return
    hs = [s[0] for s in shapes]
    ws = [s[1] for s in shapes]
    print(f"{name}: sampled {len(shapes)} images")
    print(f"  H: min={min(hs)} max={max(hs)} mean={np.mean(hs):.1f}")
    print(f"  W: min={min(ws)} max={max(ws)} mean={np.mean(ws):.1f}")

summarize_shapes(train_shapes, "Train")
summarize_shapes(val_shapes, "Val")


In [ ]:
# Random visual inspection
random.seed(0)

sample_n = 15
sample_paths = random.sample(train_imgs, k=min(sample_n, len(train_imgs)))

plt.figure(figsize=(19, 12))
for i, p in enumerate(sample_paths):
    im = Image.open(p)
    im = ImageOps.exif_transpose(im)
    plt.subplot(3, 5, i + 1)
    plt.imshow(im)
    plt.axis("off")

plt.suptitle("Random training samples", fontsize=18)
plt.tight_layout()
plt.show()


## Training

Below is a reference training call.  
Uncomment and run when you are ready.


In [ ]:
TRAIN_PARAMS = dict(
    data=str(YAML_FILE),
    epochs=300,
    imgsz=1024,
    batch=16,
    patience=20,
    optimizer="AdamW",
    lr0=5e-4,
    lrf=0.01,
    dropout=0.1,
    device=0,
    seed=42,
    amp=True,
    workers=4,
)

# results = model.train(**TRAIN_PARAMS)


## Post-training: locate run directory

Ultralytics typically writes to `runs/detect/train*`.  
Set `RUN_DIR` to the run folder you want to analyze.


In [ ]:
RUN_DIR = DATASET_ROOT / "runs" / "detect" / "train"  # update if using train2, train3, ...
assert RUN_DIR.exists(), f"Run directory not found: {RUN_DIR}"

# List artifacts
for p in sorted(RUN_DIR.glob("*")):
    print(p.name)


In [ ]:
def read_rgb(path: Path):
    img = cv2.imread(str(path))
    if img is None:
        raise FileNotFoundError(path)
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Training summary plot
results_png = RUN_DIR / "results.png"
if results_png.exists():
    img = read_rgb(results_png)
    plt.figure(figsize=(18, 7))
    plt.imshow(img)
    plt.title("Training summary (results.png)")
    plt.axis("off")
    plt.show()
else:
    print("results.png not found in RUN_DIR")


In [ ]:
# Learning curves from results.csv (matplotlib only; no seaborn)
results_csv = RUN_DIR / "results.csv"
assert results_csv.exists(), f"results.csv not found: {results_csv}"

df = pd.read_csv(results_csv)
df.columns = df.columns.str.strip()

def plot_curve(y_train: str, y_val: str, title: str, ylim=None):
    plt.figure(figsize=(10, 3.5))
    plt.plot(df["epoch"], df[y_train], label="train")
    plt.plot(df["epoch"], df[y_val], label="val")
    plt.title(title)
    plt.xlabel("epoch")
    plt.ylabel("loss")
    if ylim is not None:
        plt.ylim(ylim)
    plt.legend()
    plt.tight_layout()
    plt.show()

plot_curve("train/box_loss", "val/box_loss", "Box loss")
plot_curve("train/cls_loss", "val/cls_loss", "Classification loss")
plot_curve("train/dfl_loss", "val/dfl_loss", "DFL loss")


In [ ]:
# Curve images (if present)
curve_files = {
    "P_curve.png": "Precision–confidence curve",
    "R_curve.png": "Recall–confidence curve",
    "F1_curve.png": "F1–confidence curve",
    "PR_curve.png": "Precision–recall curve",
}

present = [f for f in curve_files if (RUN_DIR / f).exists()]
if not present:
    print("No curve images found.")
else:
    n = len(present)
    cols = 2
    rows = (n + cols - 1) // cols
    fig, axes = plt.subplots(rows, cols, figsize=(16, 5 * rows))
    axes = np.array(axes).reshape(-1)

    for ax, fname in zip(axes, present):
        img = read_rgb(RUN_DIR / fname)
        ax.imshow(img)
        ax.set_title(curve_files[fname])
        ax.axis("off")

    # Hide unused axes
    for ax in axes[len(present):]:
        ax.axis("off")

    plt.tight_layout()
    plt.show()


In [ ]:
# Confusion matrices (if present)
cm_path = RUN_DIR / "confusion_matrix.png"
cmn_path = RUN_DIR / "confusion_matrix_normalized.png"

if cm_path.exists() and cmn_path.exists():
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    axes[0].imshow(read_rgb(cm_path))
    axes[0].set_title("Confusion matrix")
    axes[0].axis("off")

    axes[1].imshow(read_rgb(cmn_path))
    axes[1].set_title("Normalized confusion matrix")
    axes[1].axis("off")

    plt.tight_layout()
    plt.show()
else:
    print("Confusion matrix images not found.")


## Evaluation & inference


In [ ]:
best_weights = RUN_DIR / "weights" / "best.pt"
assert best_weights.exists(), f"best.pt not found: {best_weights}"

best_model = YOLO(str(best_weights))


In [ ]:
# Validation on the validation split
metrics = best_model.val(split="val")

metrics_df = pd.DataFrame.from_dict(metrics.results_dict, orient="index", columns=["value"])
metrics_df


In [ ]:
# Inference grid on validation images
val_paths = list_images(VAL_IMAGES)
assert len(val_paths) > 0, f"No images found in: {VAL_IMAGES}"

# pick evenly spaced samples
k = 9
idxs = np.linspace(0, len(val_paths) - 1, num=min(k, len(val_paths)), dtype=int)
selected = [val_paths[i] for i in idxs]

fig, axes = plt.subplots(3, 3, figsize=(18, 18))
axes = np.array(axes).reshape(-1)

for ax, p in zip(axes, selected):
    pred = best_model.predict(source=str(p), imgsz=1024, verbose=False)
    annotated = pred[0].plot()
    annotated = cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB)
    ax.imshow(annotated)
    ax.set_title(p.name, fontsize=10)
    ax.axis("off")

for ax in axes[len(selected):]:
    ax.axis("off")

plt.tight_layout()
plt.show()


## Export


In [ ]:
# Export to ONNX (writes next to weights by default)
best_model.export(format="onnx")
